# SST-2
# ESN Bayesian Optimization (``ax-platform``) + Simulations

## Librairies

In [ ]:
# !pip install transformers==4.8.2
# !pip install datasets==1.7.0
# !pip install ax-platform==0.1.20

In [ ]:
import os
import sys
sys.path.insert(0, os.path.abspath("../..")) # comment this if library is pip installed

In [2]:
import io
import re
import pickle
from timeit import default_timer as timer

from tqdm.notebook import tqdm

import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

from datasets import load_dataset, Dataset, concatenate_datasets
from transformers import AutoTokenizer
from transformers import BertModel
from transformers.data.data_collator import DataCollatorWithPadding

from ax import optimize
from ax.plot.contour import plot_contour
from ax.plot.trace import optimization_trace_single_method
from ax.service.managed_loop import optimize
from ax.utils.notebook.plotting import render, init_notebook_plotting

import esntorch.core.reservoir as res
import esntorch.core.learning_algo as la
import esntorch.core.merging_strategy as ms
import esntorch.core.esn as esn

In [3]:
%config Completer.use_jedi = False
%load_ext autoreload
%autoreload 2

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [5]:
SEED = 42

## Global variables

In [6]:
pooling_strategy = 'mean' # 'mean' or 'last'

In [7]:
RESULTS_PATH = 'Results/ESN_v2' # put your path here
CACHE_DIR = '/huggingface_datasets' # put your path here
PARAMS_FILE = 'sst-2_params_uniform_'+pooling_strategy+'-pooling.pkl'
RESULTS_FILE = 'sst-2_results_esn_uniform_'+pooling_strategy+'-pooling_FINAL.pkl'

## Dataset

In [ ]:
# download dataset

# full train, mini train, and val sets
raw_datasets = load_dataset('glue', 'sst2', cache_dir=CACHE_DIR)
raw_datasets = raw_datasets.rename_column('sentence', 'text')

full_train_dataset = raw_datasets['train']
train_dataset = full_train_dataset.train_test_split(train_size=0.3, shuffle=True)['train']

val_dataset = raw_datasets['validation']

# special test set
test_dataset = load_dataset('gpt3mix/sst2', split='test', cache_dir=CACHE_DIR)

def clean(example):
    example['text'] = example['text'].replace('-LRB-', '(').replace('-RRB-', ')').replace(r'\/', r'/')
    example['label'] = np.abs(example['label'] - 1) # revert labels of test set
    return example

test_dataset = test_dataset.map(clean)

# create dataset_d
dataset_d = {}

dataset_d = {
    'full_train': full_train_dataset,
    'train': train_dataset,
    'val': val_dataset,
    'test': test_dataset
    }

In [9]:
dataset_d

{'full_train': Dataset({
     features: ['idx', 'label', 'text'],
     num_rows: 67349
 }),
 'train': Dataset({
     features: ['idx', 'label', 'text'],
     num_rows: 20204
 }),
 'val': Dataset({
     features: ['idx', 'label', 'text'],
     num_rows: 872
 }),
 'test': Dataset({
     features: ['text', 'label'],
     num_rows: 1821
 })}

In [ ]:
# tokenize

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding=False, truncation=True, return_length=True)

for k, v in dataset_d.items():
    tmp = v.map(tokenize_function, batched=True)
    tmp = tmp.rename_column('length', 'lengths')
    tmp = tmp.sort("lengths")
    tmp = tmp.rename_column('label', 'labels')
    tmp.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels', 'lengths'])
    
    dataset_d[k] = tmp

# dataloaders

dataloader_d = {}

for k, v in dataset_d.items():
    dataloader_d[k] = torch.utils.data.DataLoader(v, batch_size=256, collate_fn=DataCollatorWithPadding(tokenizer))

In [11]:
dataset_d

{'full_train': Dataset({
     features: ['attention_mask', 'idx', 'input_ids', 'labels', 'lengths', 'text', 'token_type_ids'],
     num_rows: 67349
 }),
 'train': Dataset({
     features: ['attention_mask', 'idx', 'input_ids', 'labels', 'lengths', 'text', 'token_type_ids'],
     num_rows: 20204
 }),
 'val': Dataset({
     features: ['attention_mask', 'idx', 'input_ids', 'labels', 'lengths', 'text', 'token_type_ids'],
     num_rows: 872
 }),
 'test': Dataset({
     features: ['attention_mask', 'input_ids', 'labels', 'lengths', 'text', 'token_type_ids'],
     num_rows: 1821
 })}

In [12]:
dataloader_d

{'full_train': <torch.utils.data.dataloader.DataLoader at 0x7f5ced6cddc0>,
 'train': <torch.utils.data.dataloader.DataLoader at 0x7f5cec97b490>,
 'val': <torch.utils.data.dataloader.DataLoader at 0x7f5cec97b7c0>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x7f5cec97b850>}

## Optimization

- Parameters

In [13]:
ESN_PARAMS = {
            'embedding': 'bert-base-uncased', # TEXT.vocab.vectors,
            'distribution' : 'uniform',               # uniform, gaussian
            'input_dim' : 768,                        # dim of encoding!
            'reservoir_dim' : None,
            'bias_scaling' : 0.0,
            'sparsity' : None,
            'spectral_radius' : None,
            'leaking_rate': None,
            'activation_function' : None,
            'input_scaling' : None,
            'mean' : 0.0,
            'std' : 1.0,
            'learning_algo' : None,
            'criterion' : None,
            'optimizer' : None,
            'merging_strategy' : pooling_strategy,
            'lexicon' : None,
            'bidirectional' : False, # False
            'mode' : None,
            'device' : device,
            'seed' : None
             }

In [14]:
def plug_parameters(args_d):
    
    params_d = ESN_PARAMS.copy()
    
    for k, v in args_d.items():
        if k in params_d.keys():
            params_d[k] = v
            
    return params_d

In [15]:
# args_d = {'reservoir_dim': 1000, 
#      'spectral_radius': 0.9, 
#      'leaking_rate': 0.5, 
#      'input_scaling': 0.1, 
#      'sparsity': 0.99, 
#      'activation_function': 'tanh', 
#      'alpha': 1.0}

# d = plug_parameters(args_d)

# d

- Fitness function

In [16]:
def warm_up(ESN, dataset_d):
    """Warm up the ESN."""
    
    nb_sentences = 10
    
    for i in range(nb_sentences): 

        sentence = dataset_d["train"].select([i])
        dataloader_tmp = torch.utils.data.DataLoader(sentence, 
                                                     batch_size=1, 
                                                     collate_fn=DataCollatorWithPadding(tokenizer))  

        for sentence in dataloader_tmp:
            
            ESN.warm_up(sentence)

In [17]:
def fitness(reservoir_dim=1000,
            spectral_radius=0.9,
            leaking_rate=0.5,
            input_scaling=0.1, 
            sparsity=0.99,
            activation_function='tanh',
            alpha=1.0,
            dataset_d=dataset_d, 
            dataloader_d=dataloader_d, 
            seed_l=[1234321, 420, 888],#, 1979, 7], # 5 seeds
            return_test_acc=False):
    
    # XXX
    if return_test_acc:
        seed_l=[1654547824, 1947, 1949, 1979, 1083, 1234321, 7, 42, 888, 5998989775] # 10 seeds
    elif not return_test_acc:
        seed_l=[1234321, 420, 888] # 3 seeds
    print('SEED:', seed_l)
    # XXX
    
    # create esn parameters
    args_d = locals()
    esn_params = plug_parameters(args_d)
        
    # BO mode: param optimization (esn_params['mode'] = 'esn')
    if not return_test_acc:
    
        for seed in tqdm(seed_l):
            
            acc_l = []
            
            # model
            esn_params['mode'] = 'esn'
            esn_params['seed'] = seed
            print('params:', esn_params)
            ESN = esn.EchoStateNetwork(**esn_params)
            ESN.learning_algo = la.RidgeRegression(alpha = alpha)
            ESN = ESN.to(device)

            # warm up
            warm_up(ESN, dataset_d)
            
            # train
            LOSS = ESN.fit(dataloader_d["train"])                           # mini train set
            acc = ESN.predict(dataloader_d["val"], verbose=False)[1].item() # mini val set
            acc_l.append(acc)
            
            # clean objects
            del ESN.learning_algo
            del ESN.criterion
            del ESN.merging_strategy
            del ESN
            torch.cuda.empty_cache()
            
        # results
        return np.mean(acc_l) 
          
    # prediction mode: once params have been optimized
    elif return_test_acc:
        
        # esn
        acc_esn_l = []
        time_esn_l = []
        # custom baseline
        acc_bs_l = []
        time_bs_l = []
        
        for seed in tqdm(seed_l):
            
            for mode in tqdm(['esn', 'linear_layer']):
            
                # model
                esn_params['mode'] = mode
                esn_params['seed'] = seed
                print('params:', esn_params)
                ESN = esn.EchoStateNetwork(**esn_params)
                ESN.learning_algo = la.RidgeRegression(alpha = alpha)
                ESN = ESN.to(device)
                print("RES_DIM:", ESN.reservoir.reservoir_dim)

                # warm up
                warm_up(ESN, dataset_d)

                # train
                t0 = timer()
                LOSS = ESN.fit(dataloader_d["full_train"])                       # full train set
                t1 = timer()
                acc = ESN.predict(dataloader_d["test"], verbose=False)[1].item() # full test set
                
                # clean objects
                del ESN.learning_algo
                del ESN.criterion
                del ESN.merging_strategy
                del ESN
                torch.cuda.empty_cache()
                
                # results
                if mode == 'esn':
                    time_esn_l.append(t1 - t0)
                    acc_esn_l.append(acc)
                elif mode == 'linear_layer':
                    time_bs_l.append(t1 - t0)
                    acc_bs_l.append(acc)
            
        results_d = {}
        results_d['esn'] = [np.mean(acc_esn_l), np.std(acc_esn_l), np.mean(time_esn_l), np.std(time_esn_l)]
        results_d['linear_layer'] = [np.mean(acc_bs_l), np.std(acc_bs_l), np.mean(time_bs_l), np.std(time_bs_l)]
            
        return results_d

In [18]:
# %%time

# fitness(
#         reservoir_dim=1000,
#         spectral_radius=0.9,
#         leaking_rate=0.5,
#         input_scaling=0.1, 
#         sparsity=0.99,
#         activation_function='tanh',
#         alpha=1.0,
#         dataset_d=dataset_d, 
#         dataloader_d=dataloader_d,
#         seed_l=[1234321, 7],
#         return_test_acc=True
# )

In [19]:
def wrapped_fitness(d, return_test_acc=False):
    
    return fitness(reservoir_dim=d['reservoir_dim'], # will be in the loop 
                   spectral_radius=d['spectral_radius'],
                   leaking_rate=d['leaking_rate'],
                   input_scaling=d['input_scaling'],
                   sparsity=d['sparsity'],
                   activation_function=d['activation_function'],
                   alpha=d['alpha'],
                   dataset_d=dataset_d,
                   dataloader_d=dataloader_d,
                   return_test_acc=return_test_acc)

- Otpimization

In [20]:
best_params_d = {}

for res_dim in tqdm([500, 1000, 3000, 5000]):
    
    best_parameters, best_values, experiment, model = optimize(
            parameters=[
              {
                "name": "reservoir_dim",
                "value_type": "int",
                "type": "fixed",
                "value": res_dim,
              },
              {
                "name": "activation_function",
                "value_type": "str",
                "type": "fixed",
                "value": 'tanh',
              },
              {
                "name": "spectral_radius",
                "value_type": "float",
                "type": "choice",
                "values": [0.5, 1.0, 1.5],
              },
              {
                "name": "leaking_rate",
                "value_type": "float",
                "type": "choice",
                "values": [0.1, 0.5, 0.9],
              },
              {
                "name": "input_scaling",
                "value_type": "float",
                "type": "choice",
                "values": [0.1, 1.0],
              },
              {
                "name": "sparsity",
                "value_type": "float",
                "type": "choice",
                "values": [0.0, 0.9],
              },
              {
                "name": "alpha",
                "value_type": "float",
                "type": "choice",
                "values": [0.1, 1.0, 10.0, 100.0],
              }
            ],
            # Booth function
            evaluation_function = wrapped_fitness,
            minimize = False,
            objective_name = 'val_accuracy',
            total_trials = 30 # nb iterations here
        )
    
    # results
    best_params_d[res_dim] = {}
    best_params_d[res_dim]['best_parameters'] = best_parameters
    best_params_d[res_dim]['best_values'] = best_values
    best_params_d[res_dim]['experiment'] = experiment
    # best_params_d[res_dim]['model'] = model

  0%|          | 0/4 [00:00<?, ?it/s]

[INFO 01-19 12:52:25] ax.modelbridge.dispatch_utils: Using Sobol generation strategy.
[INFO 01-19 12:52:25] ax.service.managed_loop: Started full optimization with 30 steps.
[INFO 01-19 12:52:25] ax.service.managed_loop: Running optimization trial 1...


SEED: [1234321, 420, 888]


  0%|          | 0/3 [00:00<?, ?it/s]

params: {'embedding': 'bert-base-uncased', 'distribution': 'uniform', 'input_dim': 768, 'reservoir_dim': 500, 'bias_scaling': 0.0, 'sparsity': 0.0, 'spectral_radius': 1.0, 'leaking_rate': 0.9, 'activation_function': 'tanh', 'input_scaling': 1.0, 'mean': 0.0, 'std': 1.0, 'learning_algo': None, 'criterion': None, 'optimizer': None, 'merging_strategy': 'last', 'lexicon': None, 'bidirectional': False, 'mode': 'esn', 'device': device(type='cuda'), 'seed': 1234321}


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model downloaded: bert-base-uncased
params: {'embedding': 'bert-base-uncased', 'distribution': 'uniform', 'input_dim': 768, 'reservoir_dim': 500, 'bias_scaling': 0.0, 'sparsity': 0.0, 'spectral_radius': 1.0, 'leaking_rate': 0.9, 'activation_function': 'tanh', 'input_scaling': 1.0, 'mean': 0.0, 'std': 1.0, 'learning_algo': None, 'criterion': None, 'optimizer': None, 'merging_strategy': 'last', 'lexicon': None, 'bidirectional': False, 'mode': 'esn', 'device': device(type='cuda'), 'seed': 420}


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_71654/654947317.py", line 5, in <module>
    best_parameters, best_values, experiment, model = optimize(
  File "/opt/conda/lib/python3.8/site-packages/ax/service/managed_loop.py", line 245, in optimize
    loop.full_run()
  File "/opt/conda/lib/python3.8/site-packages/ax/service/managed_loop.py", line 178, in full_run
    self.run_trial()
  File "/opt/conda/lib/python3.8/site-packages/ax/utils/common/executils.py", line 98, in actual_wrapper
    return func(*args, **kwargs)
  File "/opt/conda/lib/python3.8/site-packages/ax/service/managed_loop.py", line 168, in run_trial
    trial.fetch_data()
  File "/opt/conda/lib/python3.8/site-packages/ax/core/base_trial.py", line 372, in fetch_data
    return self.experiment._fetch_trial_data(
  File "/opt/conda/lib/python3.8/site-pa

TypeError: object of type 'NoneType' has no len()

In [22]:
# save parameters

with open(os.path.join(RESULTS_PATH, PARAMS_FILE), 'wb') as fh:
    pickle.dump(best_params_d, fh)

In [21]:
# load results
with open(os.path.join(RESULTS_PATH, PARAMS_FILE), 'rb') as fh:
    best_params_d = pickle.load(fh)

In [22]:
PARAMS_FILE

'sst-2_params_uniform_mean-pooling.pkl'

In [23]:
best_params_d

{500: {'best_parameters': {'spectral_radius': 1.0,
   'leaking_rate': 0.5,
   'input_scaling': 0.1,
   'sparsity': 0.9,
   'alpha': 1.0,
   'reservoir_dim': 500,
   'activation_function': 'tanh'},
  'best_values': ({'val_accuracy': 86.12384796142578},
   {'val_accuracy': {'val_accuracy': nan}}),
  'experiment': SimpleExperiment(None)},
 1000: {'best_parameters': {'spectral_radius': 1.0,
   'leaking_rate': 0.5,
   'input_scaling': 1.0,
   'sparsity': 0.0,
   'alpha': 100.0,
   'reservoir_dim': 1000,
   'activation_function': 'tanh'},
  'best_values': ({'val_accuracy': 85.09173583984375},
   {'val_accuracy': {'val_accuracy': nan}}),
  'experiment': SimpleExperiment(None)},
 3000: {'best_parameters': {'spectral_radius': 1.5,
   'leaking_rate': 0.9,
   'input_scaling': 0.1,
   'sparsity': 0.0,
   'alpha': 100.0,
   'reservoir_dim': 3000,
   'activation_function': 'tanh'},
  'best_values': ({'val_accuracy': 86.00917053222656},
   {'val_accuracy': {'val_accuracy': nan}}),
  'experiment': Sim

## Simulations
- Using best paramameters

In [24]:
# results

results_d = {}

for res_dim in tqdm([500, 1000, 3000, 5000]):
    
    best_parameters = best_params_d[res_dim]['best_parameters']
    d = wrapped_fitness(best_parameters, return_test_acc=True)
    results_d[res_dim] = d
    print("Experiment finished.")

  0%|          | 0/4 [00:00<?, ?it/s]

SEED: [1654547824, 1947, 1949, 1979, 1083, 1234321, 7, 42, 888, 5998989775]


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

params: {'embedding': 'bert-base-uncased', 'distribution': 'uniform', 'input_dim': 768, 'reservoir_dim': 500, 'bias_scaling': 0.0, 'sparsity': 0.9, 'spectral_radius': 1.0, 'leaking_rate': 0.5, 'activation_function': 'tanh', 'input_scaling': 0.1, 'mean': 0.0, 'std': 1.0, 'learning_algo': None, 'criterion': None, 'optimizer': None, 'merging_strategy': 'mean', 'lexicon': None, 'bidirectional': False, 'mode': 'esn', 'device': device(type='cuda'), 'seed': 1654547824}


...

params: {'embedding': 'bert-base-uncased', 'distribution': 'uniform', 'input_dim': 768, 'reservoir_dim': 5000, 'bias_scaling': 0.0, 'sparsity': 0.0, 'spectral_radius': 1.5, 'leaking_rate': 0.5, 'activation_function': 'tanh', 'input_scaling': 0.1, 'mean': 0.0, 'std': 1.0, 'learning_algo': None, 'criterion': None, 'optimizer': None, 'merging_strategy': 'mean', 'lexicon': None, 'bidirectional': False, 'mode': 'esn', 'device': device(type='cuda'), 'seed': 5998989775}


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model downloaded: bert-base-uncased
RES_DIM: 5000
params: {'embedding': 'bert-base-uncased', 'distribution': 'uniform', 'input_dim': 768, 'reservoir_dim': 5000, 'bias_scaling': 0.0, 'sparsity': 0.0, 'spectral_radius': 1.5, 'leaking_rate': 0.5, 'activation_function': 'tanh', 'input_scaling': 0.1, 'mean': 0.0, 'std': 1.0, 'learning_algo': None, 'criterion': None, 'optimizer': None, 'merging_strategy': 'mean', 'lexicon': None, 'bidirectional': False, 'mode': 'linear_layer', 'device': device(type='cuda'), 'seed': 5998989775}


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model downloaded: bert-base-uncased
RES_DIM: 5000
Experiment finished.


In [25]:
results_d

{500: {'esn': [84.06919326782227,
   0.5360616741756111,
   52.06982941627503,
   6.977747102533408],
  'linear_layer': [84.32180328369141,
   0.49009780965916744,
   53.150108721200375,
   5.310168565907716]},
 1000: {'esn': [84.31631088256836,
   0.46415134753856835,
   52.04247763864696,
   6.0525763208924115],
  'linear_layer': [84.2394302368164,
   0.4960614296098029,
   53.45674239266664,
   5.962773198438285]},
 3000: {'esn': [85.39813232421875,
   0.41998206048299347,
   53.94094805773348,
   7.036171044348623],
  'linear_layer': [85.74409713745118,
   0.21296863357199183,
   54.05177134769038,
   7.258078904974194]},
 5000: {'esn': [85.62877502441407,
   0.4050682200037713,
   55.06949195563793,
   1.422163840737339],
  'linear_layer': [86.13399276733398,
   0.2531448124397797,
   57.37928799632937,
   5.198225473931121]}}

In [26]:
RESULTS_FILE

'sst-2_results_esn_uniform_mean-pooling_FINAL.pkl'

In [27]:
# save results

with open(os.path.join(RESULTS_PATH, RESULTS_FILE), 'wb') as fh:
    pickle.dump(results_d, fh)

In [38]:
# # load results
# with open(os.path.join(RESULTS_PATH, RESULTS_FILE), 'rb') as fh:
#     results_d = pickle.load(fh)

In [ ]:
# results_d